In [1]:
# Import libs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import PIL.Image as Image
from IPython.display import display
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
available_models = [
    k for k, v in torchvision.models.__dict__.items()
    if callable(v) and k[0].islower() and k[0] != "_"
]

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Loading dataset
dataset_dir_train = "../input/training-dataset-v6/finalDataset"

train_transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                 transforms.RandomRotation(20),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


dataset = torchvision.datasets.ImageFolder(root=dataset_dir_train, transform = train_transform)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True, num_workers = 2)

In [5]:
# function to train the model
def TrainModel(model, lossFun, optimizer, scheduler, n_epochs=5):
    losses = []
    accuracies = []
    
    model.train()
    
    for epoch in range(n_epochs):
        since = time.time() # record the start time
        running_loss = 0.0
        running_correct = 0.0
        counter = 0
        for i, data in enumerate(trainloader, 0):
            
            inputs, labels = data
            # Asigning GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            counter += len(inputs)
            outputs = model(inputs)
            m = nn.LogSoftmax(dim=1)
            _, predicted = torch.max(outputs.data, 1)
            loss = lossFun(m(outputs), labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        timeSpent = time.time()-since # time used
        epoch_loss = running_loss/counter
        print(running_correct, counter)
        epoch_acc = 100*(running_correct/counter)
        print("Epoch %s, Time Used: %d s, loss: %.4f, acc: %.4f" % (epoch+1, timeSpent, epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        scheduler.step(epoch_acc)
    print('Training Finished')
    return model, losses, accuracies

In [6]:
model = models.regnet_y_3_2gf(pretrained = True)

model.fc = nn.Sequential(
    model.fc,
    nn.Linear(in_features=1000, out_features=400, bias = True),
    nn.Linear(in_features=400, out_features=150, bias=True),
    nn.Linear(in_features=150, out_features=23, bias=True)
)

model = model.to(device)

lossFun = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, threshold = 0.9)

Downloading: "https://download.pytorch.org/models/regnet_y_3_2gf-b5a9779c.pth" to /root/.cache/torch/hub/checkpoints/regnet_y_3_2gf-b5a9779c.pth


  0%|          | 0.00/74.6M [00:00<?, ?B/s]

In [7]:
model, training_losses, training_accs = TrainModel(model, lossFun, optimizer, lrscheduler, n_epochs=25)

30283.0 46374
Epoch 1, Time Used: 687 s, loss: 0.0341, acc: 65.3017
38032.0 46374
Epoch 2, Time Used: 589 s, loss: 0.0174, acc: 82.0115
40010.0 46374
Epoch 3, Time Used: 590 s, loss: 0.0131, acc: 86.2768
41266.0 46374
Epoch 4, Time Used: 587 s, loss: 0.0107, acc: 88.9852
44298.0 46374
Epoch 5, Time Used: 594 s, loss: 0.0041, acc: 95.5234
44927.0 46374
Epoch 6, Time Used: 597 s, loss: 0.0028, acc: 96.8797
45164.0 46374
Epoch 7, Time Used: 595 s, loss: 0.0023, acc: 97.3908
45363.0 46374
Epoch 8, Time Used: 593 s, loss: 0.0019, acc: 97.8199
45415.0 46374
Epoch 9, Time Used: 595 s, loss: 0.0018, acc: 97.9320
45430.0 46374
Epoch 10, Time Used: 588 s, loss: 0.0018, acc: 97.9644
45519.0 46374
Epoch 11, Time Used: 580 s, loss: 0.0017, acc: 98.1563
45462.0 46374
Epoch 12, Time Used: 579 s, loss: 0.0017, acc: 98.0334
45504.0 46374
Epoch 13, Time Used: 581 s, loss: 0.0017, acc: 98.1239
45451.0 46374
Epoch 14, Time Used: 580 s, loss: 0.0017, acc: 98.0097
45476.0 46374
Epoch 15, Time Used: 578 s, l

In [8]:
model.eval()

RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): ConvNormActivation(
          (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): ConvNormActivation(
            (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): ConvNormActivation(
            (0): Conv2d(72, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
            (1): B

In [9]:
torch.save(model, "file.pt")